# Testing Adaptation of Model

### Import Modules

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('default', category=DeprecationWarning, module='.*/qinfer/.*')
from __future__ import division # Ensures that a/b is always a float.

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import qinfer as qi
import models as m

### Setup

Plotting and exporting. Change the `overwrite` flag if you want regenerated figures to be saved to disk.

In [3]:
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

SIZE = 14
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)     # fontsize of the x any y labels
plt.rc('xtick', labelsize=10)      # fontsize of the tick labels
plt.rc('ytick', labelsize=10)      # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)   # size of the figure title

overwrite = True
def export_figure(fig, name, extensions=['.png', '.pdf']):
    for ext in extensions:
        fname = '../fig/' + name + ext
        if not os.path.isfile(fname):
            print('Saving {}'.format(fname))
            fig.savefig(fname)
        elif overwrite:
            print('Overwriting {}'.format(fname))
            fig.savefig(fname)
        else:
            print('Skipping {}'.format(fname))

### Initialize QInfer Objects

In [6]:
ham_model = m.RabiRamseyModel()

Model parameters:
   - 0: :math:`\Omega`, Rabi strength (MHz); coefficient of Sx
   - 1: :math:`\omega_e`, Zeeman frequency (MHz); coefficient of Sz
   - 2: :math:`\Delta \omega_c`, ZFS detuning (MHz); coefficient of Sz^2
   - 3: :math:`\A_N`, Nitrogen hyperfine splitting (MHz); modeled as incoherent average  
   - 4: :math:`T_2^-1`, inverse of electron T2* (MHz)
   - 5: :math:`\Omega_\text{Ramsey}`, the Rabi strength (MHz) while doing a Ramsey experiment

Experiment parameters:
 - mode: Specifies whether a reference or signal count is being performed.
 - t:   Pulse width
 - tau: Ramsey wait time (only relevent if mode is `RabiRamseyModel.RAMSEY`)
 - phi: Ramsey phase between pulses (")

In [7]:
model_prior = qi.UniformDistribution(np.array([
            [0,10],
            [0,10],
            [-5,5],
            [1.5,3.5],
            [100**-1,1**-1],
            [0,0]
        ]))

In [11]:
updater = qi.SMCUpdater(ham_model, 5000, model_prior)

In [25]:
n_rabi, n_ramsey = 51, 51
rabi_eps = np.array([
    np.linspace(0, 0.3, n_rabi),
    np.zeros(n_rabi),
    np.zeros(n_rabi),
    np.ones(n_rabi) * ham_model.RABI
]).T
rabi_eps = np.array(list(zip(*rabi_eps.T)), dtype=ham_model.expparams_dtype)

ramsey_eps = np.array([
    np.zeros(n_ramsey),
    np.linspace(0, 2, n_ramsey),
    np.zeros(n_ramsey),
    np.ones(n_rabi) * ham_model.RAMSEY
]).T
ramsey_eps = np.array(list(zip(*ramsey_eps.T)), dtype=ham_model.expparams_dtype)

all_eps = np.concatenate([rabi_eps, ramsey_eps])

### Speed Tests

In [28]:
%timeit -n1 -r1 ham_model.likelihood(1, model_prior.sample(1000), all_eps)

1 loop, best of 1: 15.7 s per loop
